In [8]:
from pathlib import Path
import pandas as pd
import numpy as np

ROOT = Path.home() / "datasets"/ "oxford-iiit-pet"

In [26]:
nono = ["b", "d"]
df = (pd.DataFrame({"name": ["a", "b", "c", "d", "e"]})
        .assign(score = np.random.randint(0, 10)))
#df = df[~df.name.apply(lambda x: x in nono)]
#df = df.drop(df[df["name"] in nono].index, axis = 0, inplace = True)
df

name     False
score    False
dtype: bool

In [2]:
BAD_IMAGES = list()
NUM_IMAGES_PER_CLASS = 200
random_state = 42
eval_split = .3

def assign_train_test_val_splits(df: pd.DataFrame) -> pd.DataFrame:
        df = df.assign(name = lambda df: df.image_path.apply(
            lambda x: "_".join(x.stem.split("_")[:-1])
        ))

        NUM_SAMPLES_PER_CLASS = int(NUM_IMAGES_PER_CLASS * eval_split)
        test = (df
                .groupby("name", group_keys=False)
                .apply(
                    lambda x: x.sample(
                    n = NUM_SAMPLES_PER_CLASS, 
                    random_state = random_state,
                    axis = 0)
                .assign(split = "test")))

        val = (df
                .drop(test.index, axis = 0)
                .groupby("name", group_keys=False)
                .apply( 
                    lambda x: x.sample( 
                    n = NUM_SAMPLES_PER_CLASS, 
                    random_state = random_state,
                    axis = 0)
                .assign(split = "val")))

        train = (df
                  .drop(test.index, axis = 0)
                  .drop(val.index, axis = 0)
                  .assign(split = "train"))

        return (pd
                .concat([train, val, test])
                .reset_index(drop = True)
                .drop("name", axis = 1))

In [3]:
df = (
    pd.DataFrame({"image_path": (ROOT/"images").rglob("*.jpg")})
    .assign(image_path = lambda df: df.image_path.apply(
        lambda x: Path(x.parent.stem,x.stem)))
    .assign(mask_path = lambda df: df.image_path.apply(
        lambda x: Path("annotations", "trimaps", f"{x.stem}.png")))
    .assign(label_str = lambda df: df.image_path.apply(
        lambda x: x.stem.split("_")[0].lower()))
    .pipe(assign_train_test_val_splits)
    )
df

,image_path,mask_path,label_str,split
0,images/pug_159,annotations/trimaps/pug_159.png,pug,train
1,images/Bombay_58,annotations/trimaps/Bombay_58.png,bombay,train
2,images/shiba_inu_32,annotations/trimaps/shiba_inu_32.png,shiba,train
3,images/pug_36,annotations/trimaps/pug_36.png,pug,train
4,images/Sphynx_186,annotations/trimaps/Sphynx_186.png,sphynx,train
...,...,...,...,...
7385,images/yorkshire_terrier_143,annotations/trimaps/yorkshire_terrier_143.png,yorkshire,test
7386,images/yorkshire_terrier_1,annotations/trimaps/yorkshire_terrier_1.png,yorkshire,test
7387,images/yorkshire_terrier_150,annotations/trimaps/yorkshire_terrier_150.png,yorkshire,test
7388,images/yorkshire_terrier_172,annotations/trimaps/yorkshire_terrier_172.png,yorkshire,test
